In [1]:
pip install tensorboard pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt

In [5]:
# 替换为你自己的 log_dir 路径
log_dir = "./logs/T16_b16_adam_lr0.001_c128_amp_cupy_Base"


ea = event_accumulator.EventAccumulator(log_dir)
ea.Reload()

# 提取 scalars
train_loss = ea.Scalars('train_loss')
train_acc = ea.Scalars('train_acc')
test_loss = ea.Scalars('test_loss')
test_acc = ea.Scalars('test_acc')
train_speed = ea.Scalars('train_speed')
test_speed = ea.Scalars('test_speed')

# 转换为 numpy 数组
epochs = [x.step for x in train_loss]
train_loss = [x.value for x in train_loss]
train_acc = [x.value for x in train_acc]
test_loss = [x.value for x in test_loss]
test_acc = [x.value for x in test_acc]
train_speed = [x.value for x in train_speed]
test_speed = [x.value for x in test_speed]

# 绘图
# ...existing code...

plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, test_loss, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

plt.subplot(1, 3, 2)
plt.plot(epochs, train_acc, label='Train Accuracy')
plt.plot(epochs, test_acc, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy Curve')

plt.subplot(1, 3, 3)
plt.plot(epochs, train_speed, label='Train Speed')
plt.plot(epochs, test_speed, label='Test Speed')
plt.xlabel('Epoch')
plt.ylabel('Images/s')
plt.legend()
plt.title('Speed Curve')

plt.tight_layout()
plt.show()


KeyError: 'Key train_speed was not found in Reservoir'